# Predictive Modelling of Claim Frequency and Severity

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scikit-learn as skl

SyntaxError: invalid syntax (4079712538.py, line 4)